In [1]:
import cv2 # OpenCV
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import pandas as pd

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path = "D:\FULL"

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.5)
training_set = train_datagen.flow_from_directory(path+"/train",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
validation_set = train_datagen.flow_from_directory(path+"/test",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,

                                                class_mode='categorical',
                                                subset='training')

Found 15923 images belonging to 7 classes.
Found 3976 images belonging to 7 classes.


In [5]:
resnetv2 = keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
# KALO FULL DIGANTI FALSE 
resnetv2.trainable = True
print(len(resnetv2.layers))

192


In [6]:
i = 0
for layer in resnetv2.layers:
  if i > 170:
    layer.trainable = False
  i+=1

In [7]:
#Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

input = tf.keras.layers.Input(shape=(224, 224, 3))
# Apply data augmentation to the input
x = data_augmentation(input)
x = resnetv2(input, training=False)
# x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
resnetv2 = tf.keras.models.Model(inputs = input, outputs = x, name = "resnetv2")

resnetv2.summary()

Model: "resnetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 1000)           │    25,613,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         7,007 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,620,807 (97.74 MB)

 Trainable params: 15,646,687 (59.69 MB)

 Non-trainable params: 9,974,120 (38.05 MB)

In [8]:
training_set.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'suprise': 6}

In [9]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/10


c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 1143s 18s/step - accuracy: 0.1836 - loss: 1.9342 - val_accuracy: 0.1719 - val_loss: 1.9181
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1051s 17s/step - accuracy: 0.1911 - loss: 1.9123 - val_accuracy: 0.2344 - val_loss: 1.8996
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1054s 17s/step - accuracy: 0.2540 - loss: 1.8935 - val_accuracy: 0.2260 - val_loss: 1.8886
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1089s 18s/step - accuracy: 0.2435 - loss: 1.8818 - val_accuracy: 0.2229 - val_loss: 1.8867
Epoch 5/10
 1/62 ━━━━━━━━━━━━━━━━━━━━ 18:10 18s/step - accuracy: 0.1875 - loss: 1.8758

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 28s 168ms/step - accuracy: 0.1875 - loss: 1.8758 - val_accuracy: 0.2721 - val_loss: 1.8481
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1082s 17s/step - accuracy: 0.2313 - loss: 1.8766 - val_accuracy: 0.2031 - val_loss: 1.8782
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1056s 17s/step - accuracy: 0.2337 - loss: 1.8730 - val_accuracy: 0.2240 - val_loss: 1.8743
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1054s 17s/step - accuracy: 0.2354 - loss: 1.8635 - val_accuracy: 0.2458 - val_loss: 1.8676
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1056s 17s/step - accuracy: 0.2464 - loss: 1.8538 - val_accuracy: 0.2427 - val_loss: 1.8636
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 29s 155ms/step - accuracy: 0.2656 - loss: 1.8408 - val_accuracy: 0.1985 - val_loss: 1.8657


In [11]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1145s 17s/step - accuracy: 0.2369 - loss: 1.8525 - val_accuracy: 0.2313 - val_loss: 1.8609
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1078s 17s/step - accuracy: 0.2407 - loss: 1.8526 - val_accuracy: 0.2323 - val_loss: 1.8584
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1031s 17s/step - accuracy: 0.2295 - loss: 1.8478 - val_accuracy: 0.2292 - val_loss: 1.8554
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1027s 17s/step - accuracy: 0.2578 - loss: 1.8379 - val_accuracy: 0.2583 - val_loss: 1.8379
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 29s 151ms/step - accuracy: 0.2031 - loss: 1.8449 - val_accuracy: 0.2574 - val_loss: 1.8449
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1113s 18s/step - accuracy: 0.2478 - loss: 1.8521 - val_accuracy: 0.2396 - val_loss: 1.8462
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1043s 17s/step - accuracy: 0.2401 - loss: 1.8532 - val_accuracy: 0.2500 - val_loss: 1.8375
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1030s 17s/step - accuracy: 0.2382 - loss: 1.8499 - val_accu

In [12]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=40,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1135s 17s/step - accuracy: 0.2361 - loss: 1.8458 - val_accuracy: 0.2469 - val_loss: 1.8362
Epoch 2/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1044s 17s/step - accuracy: 0.2302 - loss: 1.8522 - val_accuracy: 0.2531 - val_loss: 1.8534
Epoch 3/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1046s 17s/step - accuracy: 0.2349 - loss: 1.8494 - val_accuracy: 0.2375 - val_loss: 1.8563
Epoch 4/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1033s 17s/step - accuracy: 0.2357 - loss: 1.8420 - val_accuracy: 0.2583 - val_loss: 1.8172
Epoch 5/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 27s 161ms/step - accuracy: 0.2031 - loss: 1.8359 - val_accuracy: 0.1985 - val_loss: 1.8512
Epoch 6/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1032s 17s/step - accuracy: 0.2399 - loss: 1.8455 - val_accuracy: 0.2271 - val_loss: 1.8566
Epoch 7/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1055s 17s/step - accuracy: 0.2414 - loss: 1.8549 - val_accuracy: 0.2479 - val_loss: 1.8459
Epoch 8/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1090s 18s/step - accuracy: 0.2608 - loss: 1.8465 - val_accu

In [10]:
resnetv2.save("resnetv2-lbh-1.keras")

In [13]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1159s 18s/step - accuracy: 0.2329 - loss: 1.8496
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1098s 18s/step - accuracy: 0.2475 - loss: 1.8366
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1094s 18s/step - accuracy: 0.2461 - loss: 1.8410
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1100s 18s/step - accuracy: 0.2448 - loss: 1.8367
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - accuracy: 0.2031 - loss: 1.8613 
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1113s 18s/step - accuracy: 0.2428 - loss: 1.8414
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1108s 18s/step - accuracy: 0.2326 - loss: 1.8483
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1088s 18s/step - accuracy: 0.2434 - loss: 1.8515
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1083s 17s/step - accuracy: 0.2500 - loss: 1.8313
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.2344 - loss: 1.8254 


In [14]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=40,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1383s 21s/step - accuracy: 0.2458 - loss: 1.8280 - val_accuracy: 0.2313 - val_loss: 1.8544
Epoch 2/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1407s 23s/step - accuracy: 0.2351 - loss: 1.8413 - val_accuracy: 0.2313 - val_loss: 1.8405
Epoch 3/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1261s 20s/step - accuracy: 0.2361 - loss: 1.8475 - val_accuracy: 0.2385 - val_loss: 1.8461
Epoch 4/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1199s 19s/step - accuracy: 0.2239 - loss: 1.8428 - val_accuracy: 0.2438 - val_loss: 1.8294
Epoch 5/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 28s 147ms/step - accuracy: 0.2656 - loss: 1.8201 - val_accuracy: 0.2868 - val_loss: 1.7992
Epoch 6/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1189s 19s/step - accuracy: 0.2444 - loss: 1.8391 - val_accuracy: 0.2208 - val_loss: 1.8617
Epoch 7/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1161s 19s/step - accuracy: 0.2313 - loss: 1.8408 - val_accuracy: 0.2469 - val_loss: 1.8491
Epoch 8/40
62/62 ━━━━━━━━━━━━━━━━━━━━ 1180s 19s/step - accuracy: 0.2348 - loss: 1.8513 - val_accu

In [6]:
new_resnetv2_lbh = tf.keras.models.load_model("resnetv2-lbh-110.keras")

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 316 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

new_resnetv2_lbh.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = new_resnetv2_lbh.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=30,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/30


c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 901s 14s/step - accuracy: 0.2299 - loss: 1.8437 - val_accuracy: 0.2479 - val_loss: 1.8484
Epoch 2/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 876s 14s/step - accuracy: 0.2522 - loss: 1.8425 - val_accuracy: 0.2385 - val_loss: 1.8435
Epoch 3/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 867s 14s/step - accuracy: 0.2377 - loss: 1.8507 - val_accuracy: 0.2333 - val_loss: 1.8637
Epoch 4/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 863s 14s/step - accuracy: 0.2557 - loss: 1.8398 - val_accuracy: 0.2396 - val_loss: 1.8358
Epoch 5/30
 1/62 ━━━━━━━━━━━━━━━━━━━━ 15:12 15s/step - accuracy: 0.2188 - loss: 1.8736

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 23s 133ms/step - accuracy: 0.2188 - loss: 1.8736 - val_accuracy: 0.2794 - val_loss: 1.8531
Epoch 6/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 867s 14s/step - accuracy: 0.2444 - loss: 1.8461 - val_accuracy: 0.2469 - val_loss: 1.8457
Epoch 7/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 861s 14s/step - accuracy: 0.2522 - loss: 1.8347 - val_accuracy: 0.2479 - val_loss: 1.8358
Epoch 8/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 866s 14s/step - accuracy: 0.2389 - loss: 1.8568 - val_accuracy: 0.2417 - val_loss: 1.8475
Epoch 9/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 865s 14s/step - accuracy: 0.2324 - loss: 1.8560 - val_accuracy: 0.2646 - val_loss: 1.8421
Epoch 10/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 22s 111ms/step - accuracy: 0.3438 - loss: 1.7340 - val_accuracy: 0.2574 - val_loss: 1.8645
Epoch 11/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 861s 14s/step - accuracy: 0.2321 - loss: 1.8508 - val_accuracy: 0.2177 - val_loss: 1.8670
Epoch 12/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 871s 14s/step - accuracy: 0.2318 - loss: 1.8469 - val_accuracy: 0.2781 -

In [8]:
new_resnetv2_lbh.save("resnetv2-lbh-140.keras")

In [4]:
new2_resnetv2_lbh = tf.keras.models.load_model("resnetv2-lbh-140.keras")

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 316 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

new2_resnetv2_lbh.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = new2_resnetv2_lbh.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=30,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/30


c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 1358s 21s/step - accuracy: 0.2374 - loss: 1.8396 - val_accuracy: 0.2406 - val_loss: 1.8576
Epoch 2/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1359s 22s/step - accuracy: 0.2166 - loss: 1.8475 - val_accuracy: 0.2198 - val_loss: 1.8665
Epoch 3/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1220s 20s/step - accuracy: 0.2466 - loss: 1.8375 - val_accuracy: 0.2385 - val_loss: 1.8320
Epoch 4/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1247s 20s/step - accuracy: 0.2454 - loss: 1.8508 - val_accuracy: 0.2510 - val_loss: 1.8309
Epoch 5/30
 1/62 ━━━━━━━━━━━━━━━━━━━━ 21:38 21s/step - accuracy: 0.2500 - loss: 1.8161

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 32s 171ms/step - accuracy: 0.2500 - loss: 1.8161 - val_accuracy: 0.2426 - val_loss: 1.8513
Epoch 6/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1265s 20s/step - accuracy: 0.2442 - loss: 1.8384 - val_accuracy: 0.2552 - val_loss: 1.8298
Epoch 7/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1350s 22s/step - accuracy: 0.2359 - loss: 1.8463 - val_accuracy: 0.2365 - val_loss: 1.8520
Epoch 8/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1401s 23s/step - accuracy: 0.2307 - loss: 1.8493 - val_accuracy: 0.2375 - val_loss: 1.8462
Epoch 9/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1316s 21s/step - accuracy: 0.2549 - loss: 1.8414 - val_accuracy: 0.2448 - val_loss: 1.8424
Epoch 10/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 32s 155ms/step - accuracy: 0.2500 - loss: 1.7879 - val_accuracy: 0.2721 - val_loss: 1.8104
Epoch 11/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1382s 22s/step - accuracy: 0.2429 - loss: 1.8441 - val_accuracy: 0.2427 - val_loss: 1.8415
Epoch 12/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1285s 21s/step - accuracy: 0.2430 - loss: 1.8516 - val_accuracy: 0.

In [ ]:
new2_resnetv2_lbh.save("resnetv2-lbh-170.keras")